In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.auto import tqdm
import pretty_midi
import sys
np.set_printoptions(threshold=sys.maxsize)

min_drum_pitch = 35
max_drum_pitch = 81
n_drum_pitches = max_drum_pitch - min_drum_pitch + 1

min_pitch = 0
max_pitch = 127
n_pitches = max_pitch - min_pitch + 1

sequence_length = 128
n_velocities = 128
n_instruments = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from model import MusicGen
from inference import generate, tensor_to_midi
meta = torch.load("data/processed/LMD-clean-100/LMD-clean-100_meta.pth")
model = MusicGen(meta)
model.load_state_dict(torch.load('trained_weights/model12feb-model', weights_only=True))
model.to(device)

cuda


C:\Users\aniru\AppData\Local\Temp\ipykernel_8636\4079401975.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load("data/processed/LMD-clean-100/LMD-clean-10

MusicGen(
  (lstm): LSTM(5, 512, batch_first=True)
  (regular_pitch_head): Linear(in_features=512, out_features=128, bias=True)
  (drum_pitch_head): Linear(in_features=512, out_features=47, bias=True)
  (velocity_head): Linear(in_features=512, out_features=128, bias=True)
  (step_head): Linear(in_features=512, out_features=1, bias=True)
  (duration_head): Linear(in_features=512, out_features=1, bias=True)
  (instrument_head): Linear(in_features=512, out_features=4, bias=True)
)

In [2]:
loaded_data = torch.load('data/processed/LMD-clean-100/LMD-clean-100_0.pth', weights_only=True)[0][0:128]
print(loaded_data.shape)

torch.Size([128, 5])


In [ ]:
# generate seeds
seeds = []
for i in range(10):
  seed = torch.load('data/processed/LMD-clean-100/LMD-clean-100_0.pth', weights_only=True)[i][0:128]
  seeds.append(seed)

torch.save(seeds, f'seeds.pth')

In [3]:
#seed_sequence = torch.zeros((1, sequence_length, 5))
generated_notes = generate(model, loaded_data.unsqueeze(0), steps=1000, device=device)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
generated_midi = tensor_to_midi(generated_notes)
generated_midi.write("generated/v1/multi_instrument_v1_sample6.midi")